In [1]:
import csv
import pandas as pd
import collections
import codecs
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist
import pickle
from nltk.stem import WordNetLemmatizer

#nltk.download('stopwords')

In [2]:
loadPath = "/Users/ipinni/Library/CloudStorage/OneDrive-UniversityofLeeds/UKRI_Tweet_Data/completed/"
savePath = "/Users/ipinni/Library/CloudStorage/OneDrive-UniversityofLeeds/UKRI_Tweet_Data/BigramAnalysis/"

May need to edit the stopwords and run again.

In [3]:
default_stopwords_en = set(nltk.corpus.stopwords.words('english'))
my_stopwords = set(["rt","RT", "&", "amp", "&amp", "http","https", "http://", "https://", "fav", "FAV", "cop","@cop20", "@cop21", "@cop22", "@cop23", "@cop24", "@cop25","@cop26", "#cop20", "#cop21", "#cop22", "#cop23", "#cop24", "#cop25","#cop26", "20", "21","22","23","24","25","26", "u", "...", "@"])
new_stops = default_stopwords_en | my_stopwords
filter_stops = lambda w: len(w)< 2 or w in new_stops

In [4]:
tt = TweetTokenizer()     

def get_words(tweets):
    tweets_text = tweets['text'].apply(tt.tokenize)     
    flat_list = [item for sublist in tweets_text for item in sublist] 
    words = [word for word in flat_list if len(word) > 1]     
    words = [word.lower() for word in words]     
    words = [word for word in words if word not in new_stops]
    words = [word for word in words if '@' not in word]
    #lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return(words)

In [5]:
def get_topics(topics, docs, choices):
    topic_docs = {topic: [] for topic in set(topics)}
    for topic, doc in zip(topics, docs):
        topic_docs[topic].append(doc)
    
    chosen_topics = []
    for i in range(len(choices)):
        chosen_topics.extend(topic_docs[choices[i]])
    
    tweets = pd.DataFrame(chosen_topics, columns=['text'])

    return(tweets) 

Frames = red, Delay = blue, fringe = green

In [6]:
COP20_frames = [3,4,5,7,14,22,26,40,41,42,44]
#COP20_delay = [6,15,17,31,45]
#COP20_fringe = [2,26,41]

In [7]:
COP21_frames = [3,4,9,14,15,19,29]
#COP21_delay = [35,36]
#COP21_fringe = [4,12,17,22,23,26,28,34]

In [8]:
COP22_frames = [13,15,16,17,18,28]
#COP22_delay = [20,43,48,52,55]
#COP22_fringe = [1,7,8,14,15,33,50]

In [9]:
COP23_frames = [4,13,17,18,31]
#COP23_delay = []
#COP23_fringe = [1,4,5,14,26]

In [10]:
COP24_frames = [2,8,17,20,21,26,35,43,44,46,48,51,55,56]
#COP24_delay = [22]
#COP24_fringe = [13,21,23,25,28,33,34,36,38,47,48,49,54]

In [11]:
COP25_frames = [3,4,7,8,16,18,26,32,35,43,65]
#COP25_delay = [21,36,55]
#COP25_fringe = [9,12,13,15,16,19,30,32,33,34,44,48,49,62]

In [12]:
COP26_frames = [3,5,10,23,27,36,37]
#COP26_delay = [31]
#COP26_fringe = [1,3,5,6,8,15,16,18,21,23,30,34,39,40,41,42,44]

In [14]:
COP26_frames

[5, 10, 20, 23, 27, 36, 37]

In [20]:
FFF2018_frames = [i for i in range(43)]
FFF2019_frames = [i for i in range(55)]
FFF2020_frames = [i for i in range(51)]
FFF2021_frames = [i for i in range(44)]

In [ ]:

[2,6,16,23,24,29,32]
FFF2018_delay = [11,17,36,41]
FFF2018_fringe = [1,4,5,7,10,15,20,22,26,27,28,31,33,37,38,39,42]

In [15]:
FFF2019_frames = [9,12,20,38,40,47]
FFF2019_delay = [6,16,31,37,46]
FFF2019_fringe = [1,2,3,8,10,18,24,28,29,30,32,34,49,54]

In [6]:
FFF2020_frames = [10,18,19,28,33,38]
FFF2020_delay = [45]
FFF2020_fringe = [2,4,6,7,8,12,14,17,20,22,23,29,30,37,46,49]

In [7]:
FFF2021_frames = [6,14,19,21,26,27,32]
FFF2021_delay = [22,23,29,30,33,40]
FFF2021_fringe = [1,2,3,4,5,7,8,10,17,25,28,41,42,43]

Change the value of COPversion and runn all cells below

In [38]:
COPversion = "COP26"

In [39]:
def get_data():
    with open(loadPath + COPversion + "/" + COPversion + "topics.list" ,'rb') as config_list_file:   
        topics = pickle.load(config_list_file)

    with open(loadPath + COPversion + "/" + COPversion + "docs.list", 'rb') as docs_list_file:   
        docs = pickle.load(docs_list_file)

    return(topics, docs)

topics, docs = get_data()

In [40]:
frames = get_topics(topics, docs, eval((COPversion+"_frames")))


In [41]:
def biFrames(text):
    bgm = nltk.collocations.BigramAssocMeasures()
    bcf = nltk.collocations.BigramCollocationFinder.from_words(text)
    bcf.apply_word_filter(filter_stops)
    scored = bcf.score_ngrams(bgm.student_t)[:750]
    temp_dict = dict(scored)
    csvData = ["Source, Target, Weight"]
    for (col1, col2), col3 in iter(temp_dict.items()):
        csvData.append("%s, %s, %s" % (col1, col2, col3))
    f = open(savePath + "Frames/" + COPversion + 'BigramFrames' + '.csv', 'w')
    f.write("\n".join(csvData))
    f.close()
biFrames(get_words(frames))

In [ ]:
delay = get_topics(topics, docs, eval((COPversion+"_delay")))
fringe = get_topics(topics, docs, eval((COPversion+"_fringe")))

In [100]:
def biDelay(text):
    bgm = nltk.collocations.BigramAssocMeasures()
    bcf = nltk.collocations.BigramCollocationFinder.from_words(text)
    bcf.apply_word_filter(filter_stops)
    scored = bcf.score_ngrams(bgm.student_t)[:750]
    temp_dict = dict(scored)
    csvData = ["Source, Target, Weight"]
    for (col1, col2), col3 in iter(temp_dict.items()):
        csvData.append("%s, %s, %s" % (col1, col2, col3))
    f = open(savePath + "Delay/" + COPversion + 'BigramDelay' +'.csv', 'w')
    f.write("\n".join(csvData))
    f.close()
biDelay(get_words(delay))

In [101]:
def biFringe(text):
    bgm = nltk.collocations.BigramAssocMeasures()
    bcf = nltk.collocations.BigramCollocationFinder.from_words(text)
    bcf.apply_word_filter(filter_stops)
    scored = bcf.score_ngrams(bgm.student_t)[:750]
    temp_dict = dict(scored)
    csvData = ["Source, Target, Weight"]
    for (col1, col2), col3 in iter(temp_dict.items()):
        csvData.append("%s, %s, %s" % (col1, col2, col3))
    f = open(savePath + "Fringe/" + COPversion + 'BigramFringe' + '.csv', 'w')
    f.write("\n".join(csvData))
    f.close()
biFringe(get_words(fringe))